In [ ]:
##########   Importing neccessary libraries  ########

import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from scipy import stats as sp
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
import cartopy as cp
import cartopy.crs as ccrs
import cartopy.feature as cf
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import glob



<img src="images/spi_eq.png">
<img src="Web capture_27-10-2021_152920_www.youtube.com (1).jpeg.crdownload">


Where: 

N = current monthly/yearly rainfall, in order words, of the month/year when RAI will be generated (mm); 

$\overline{N}$ = monthly/yearly average rainfall of the historical series (mm); 
    
$\overline{M}$ = average of the ten highest monthly/yearly precipitations of the historical series (mm); 
    
$\overline{X}$ = average of the ten lowest monthly/ yearly precipitations of the historical series (mm); and positive anomalies have their values above average and negative anomalies have their values below average.

In [ ]:
########### Specifying the path for some data since they are not in the working directory   ###########

########### Path for Gauge Station Data ############
filename = Path('data/')

######## Path for CRU data  #######
cru_path = Path('C:/Users/Stephen Asare/Desktop/DATASETS/CRU/')


In [ ]:
#########Reading in names and locations of stations  #########

dat = pd.read_fwf('GMet_location_avgSI.txt', names = ['Station', 'Longitude', 'Latitude', 'St'])

In [ ]:
#####################  Plot for  study Area #######



plt.figure(figsize=(10,9))                         ###### Specifying the size of the figure
ax = plt.axes(projection = ccrs.PlateCarree())       ###### Specifying the type of geopatial plot
ax.add_feature(cf.COASTLINE,alpha=0.8)             
ax.add_feature(cf.BORDERS)
#ax.add_feature(cf.LAND)
ax.set_extent([-3.5,1.2,11.4,4.5])                   #### setting the map boundaries
#ax.stock_img()
ax.add_feature(cf.STATES, alpha= 0.1)               ####  adding territorial boundaries

ax.plot(dat.Longitude,                            
        dat.Latitude, 
        'ro',                                       ##### plotting the longitudes and latitudes of the station
        ms=7, 
        color = 'k')#,
        #transform=ccrs.Geodetic(),label='Synoptic stations')  

s_stations = np.asarray(dat.Station)
                          
for longitude, latitude, name in zip(dat.Longitude, dat.Latitude, s_stations):
    if name in ['Yendi']:
        ax.text(longitude - .05, latitude - .15, 
                name, 
                va='center',
                ha='center', transform=ccrs.Geodetic(), fontweight='bold',fontsize = '12')
    else:    
        ax.text(longitude + .09, latitude + .12, 
                name, 
                va='center',
                ha='center', transform=ccrs.Geodetic(), fontweight='bold',fontsize = '12')


ax.set_xticks([-3.2,-2.2,-1.2,0,1.2], crs=ccrs.PlateCarree())
ax.set_yticks([11,10,9,8,7,6,5], crs=ccrs.PlateCarree())
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)
#plt.savefig('Synoptic Stations.pdf',bbox_inches = 'tight')
plt.savefig('Ghana_map.JPEG',bbox_inches = 'tight')

In [ ]:
##############  Reading in Gridded datasets  ###########

#####  CHIRPS

da_data = xr.open_mfdataset('chirps/chirps-v2.0.*.days_p25.nc', combine = 'by_coords')
ds_data = da_data.sel(time = slice('1983','2017')).precip
gh_data = ds_data.sel(latitude = slice(4,12), longitude = slice(-3,1.5))
gh_data = gh_data.resample(time = 'y').sum('time')


######   CRU
cru = xr.open_dataset(cru_path/('cru_ts4.05.1901.2020.pre.dat.nc')).pre
cru_data = cru.sel(lat = slice(4,12), lon = slice(-3,1.5), time = slice('1983','2017'))
cru_data = cru_data.resample(time = 'y').sum('time')

In [ ]:
!pip install scipy

In [ ]:
#####Rainfall Aanomaly Index function for Station data

def RAI(x):
    anomaly = x - np.nanmean(x)
    N_bar = x.mean()
    X_bar = x.sort_values().head(10).mean()
    M_bar = x.sort_values(ascending=False).head(10).mean()
    positive = anomaly.loc[anomaly>=0]
    negative = anomaly.loc[anomaly<0]
    p=3*((positive)/(M_bar-N_bar))
    n=-3*((negative)/(X_bar-N_bar))        
    c = pd.concat([p,n]).sort_index()
    return c


######Rainfall Anomaly Index Function for Gridded data

def RAIG(data, dimension):
    overall_mean = data.mean(dimension)    
    anomaly = data - overall_mean    
    sortings = data.reduce(np.sort,dim=dimension)  
    da_lowest_10 = sortings[:10].mean(dimension)    
    da_highest_10 = sortings[:-10:-1].mean(dimension)
    negatives = -3*( (anomaly.where(anomaly<0)) / (da_lowest_10-overall_mean) ) 
    positives = 3*( (anomaly.where(anomaly>0)) / (da_highest_10-overall_mean) ) 
    RAI = anomaly.where(anomaly>=0, negatives).where(anomaly<0, positives)   
    return RAI


#######Statistics for Rainfall events

class stats:
    def hits(gauge, gridded, case):
        if case == 'drought':
            output = gauge.where((gauge<=-1) & (gridded<=-1)).count()
        elif case == 'flood':
            output = gauge.where((gauge>=1) & (gridded>=1)).count()
        return output
        
        
    def misses(gauge, gridded, case):
        if case == 'drought':
            output = gauge.where((gauge<=-1) & (gridded>-1)).count()    
                        ##### why isn't the code (Gauge.where(gauge<=-1))&(Gridded.where(dridded>=-1)    
        elif case == 'flood':
            output = gauge.where((gauge>=1) & (gridded<1)).count()
        return output
        
    def false_alarms(gauge, gridded, case):
        if case == 'drought':
            output = gauge.where((gauge>-1) & (gridded<=-1)).count()
        elif case == 'flood':
            output = gauge.where((gauge<1) & (gridded>=1)).count()
        return output
      
        
    def correct_negatives(gauge, gridded, case):
        if case == 'drought':
            output = gauge.where((gauge>-1) & (gridded>-1)).count()
        elif case == 'flood':
            output = gauge.where((gauge<1) & (gridded<1)).count()
        return output
 
    
    def POD(gauge, gridded, case):
        H = stats.hits(gauge, gridded, case)
        M = stats.misses(gauge, gridded, case)
        return H/(H+M)


    def FAR(gauge, gridded, case):
        F = stats.false_alarms(gauge, gridded, case)
        H = stats.hits(gauge, gridded, case)
        return F/(H+F)

    
        
    def nse(gauge, gridded):
        num = ((gridded-gauge)**2).sum()
        den = ((gauge-np.nanmean(gauge))**2).sum()
        return 1 - (num/den)
    
    
    def correlation(gauge, gridded):
        return sp.pearsonr(gauge, gridded)[0]
        
  
    def sedi(gauge, gridded, case):
        F = stats.false_alarms(gauge, gridded, case)
        H = stats.hits(gauge, gridded, case)
        num = np.log10(F) - np.log10(H) - np.log10(1-F) + np.log10(1-H)
        den = np.log10(F) + np.log10(H) + np.log10(1-F) + np.log10(1-H)
        return num/den

    def PC(gauge, gridded):
       
        return ((gridded - gauge) / gauge)
    
    
    def standard_error(x):
        ##### standard error function
        try:
            out = np.nanstd(x)/np.sqrt(np.size(x))
        except:
            out = x.nanstd()/np.sqrt(x.size())
        return out
    
    
    def mse(y):
        return stats.standard_error(y)**2

    
#Chances that the event will actually occur?
    def chance_occur(gauge, gridded, case):
        H = stats.hits(gauge, gridded, case)
        F = stats.false_alarms(gauge, gridded, case)
        return (H/(H+F))*100 


# When events occur, how often are the gridded (forecast) correct?
    def Often_correct(gauge, gridded, case):
        return stats.POD(gauge, gridded, case)*100
    
        
#Do the forecasts (gridded) predict events too often / not too often enough?
    def prediction(gauge, gridded, case):
        F = stats.false_alarms(gauge, gridded, case)
        H = stats.hits(gauge, gridded, case)
        M = stats.misses(gauge, gridded, case)
        return (((H+F)-(H+M))/(H+M))*100
    
def functions(a, b, c):
    POD = np.round(stats.POD(a, b, c),2)
    FAR = np.round(stats.FAR(a, b, c),2)
    nse = np.round(stats.nse(a, b), 2)
    sedi = np.round(stats.sedi(a, b, c), 2)
    r = np.round(stats.correlation(a, b), 2)
    C_H = np.round(stats.chance_occur(a, b, c), 2)
    O_F = np.round(stats.Often_correct(a, b, c), 2)
    pre = np.round(stats.prediction(a, b, c), 2)
    PC = np.round(stats.PC(a,b), 2)
    std = np.round(stats.standard_error(RAI_data.Gridded),2)
    mse = np.round(stats.mse(RAI_data.Gridded),2)
                   
    return [POD, FAR, nse, r, std, mse, C_H, O_F, pre, sedi]   


###########################
New_data_drought = {}
New_data_flood = {}
for i in range (22):
    st = dat.iloc[i,0]
    G_met =  pd.read_fwf(filename/(st+'_1983_2017_dRR_gapless.txt'), names = ['Year', 'Month', 'Day', 'Precip'], index_col = 'Year')
    G_met = G_met['Precip'].groupby('Year').sum()
    RAI_Gmet = RAI(G_met)
    
    lon = dat.iloc[i,1]
    lat = dat.iloc[i,2]
    
    st_Gridded = gh_data.sel(latitude = lat, longitude = lon, method = 'nearest')
    RAI_Grid = RAIG(st_Gridded, 'time')
    
    RAI_data= pd.DataFrame(RAI_Gmet.values, columns=['Gmet'])
    RAI_data['Gridded'] = RAI_Grid.values
    RAI_data = RAI_data.set_index(RAI_Gmet.index)
    
    x = functions(RAI_data.Gmet, RAI_data.Gridded,'drought')
    x.insert(0,lat)
    x.insert(0,lon)
    New_data_drought[st] = x
    
    z = functions(RAI_data.Gmet, RAI_data.Gridded,'flood')
    z.insert(0,lat)
    z.insert(0,lon)
    New_data_flood[st] = z
    #print(st,lon,lat,x)
#pd.DataFrame(New_data)
fl = pd.DataFrame(New_data_drought).T
fl.columns = ['Lon','Lat','POD','FAR','nse', 'r','Standard_error','MSE','Chances_occur','Often_correct','Prediction','SEDI']

dr = pd.DataFrame(New_data_flood).T
dr.columns = ['Lon','Lat','POD','FAR','nse', 'r','Standard_error','MSE','Chances_occur','Often_correct','Prediction','SEDI']


1. (H/(H+F))*100    #Chances that the event will actually occur?
2. (H/(H+M))*100    # When events occur, how often are the gridded (forecast) correct?

3. Do the forecasts (gridded) predict events too often / not too often enough?
(((H+F)-(H+M))  /(H+M))*100

In [ ]:





# fl

In [ ]:
# dr

In [ ]:
######## Plots for Statistics  ######

fig, axes = plt.subplots(nrows = 3, ncols = 3, figsize = (16,20), subplot_kw = {'projection' : ccrs.PlateCarree()})
# ax=axes.flatten()
for i in range(9):
    ax[i].add_feature(cf.COASTLINE.with_scale('10m'), linewidth=0.5)
    ax[i].add_feature(cf.BORDERS,linewidth=0.5)
    ax[i].set_extent([-3.4,1.4,11.5,4.5])
    ax[i].set_xticks([-3.2,-2.2,-1.2,0,1.2], crs=ccrs.PlateCarree())
    ax[i].set_yticks([11,10,9,8,7,6,5], crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax[i].xaxis.set_major_formatter(lon_formatter)
    ax[i].yaxis.set_major_formatter(lat_formatter)
    ax[i].set_title(fl.columns[2+i])
    
    cb = ax[i].scatter(x=fl['Lon'], y=fl['Lat'], c=fl.iloc[:,2+i], cmap='plasma',linewidths = 5,s=100)
    if i in [0,3,6]:
        fig.colorbar(cb, ax=axes[int(i/3),2])
#         plt.scatter

In [ ]:
fig, axes = plt.subplots(nrows = 3, ncols = 3, figsize = (16,20), subplot_kw = {'projection' : ccrs.PlateCarree()})
ax=axes.flatten()
for i in range(9):
    ax[i].add_feature(cf.COASTLINE.with_scale('10m'), linewidth=0.5)
    ax[i].add_feature(cf.BORDERS,linewidth=0.5)
    ax[i].set_extent([-3.4,1.4,11.5,4.5])
    ax[i].set_xticks([-3.2,-2.2,-1.2,0,1.2], crs=ccrs.PlateCarree())
    ax[i].set_yticks([11,10,9,8,7,6,5], crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax[i].xaxis.set_major_formatter(lon_formatter)
    ax[i].yaxis.set_major_formatter(lat_formatter)
    ax[i].set_title(dr.columns[2+i])
    
    cb=ax[i].scatter(x=dr['Lon'], y=dr['Lat'], c=dr.iloc[:,2+i], cmap='plasma',linewidths = 5,s=100)
    if i in [0,3,6]:
        fig.colorbar(cb, ax=axes[int(i/3),2])
        plt.scatter

In [ ]:
#########  Script for Rinfall parttern     #########


DATA = pd.DataFrame()

for i in range (22):
    
    st = dat.iloc[i,0]
    G_met =  pd.read_fwf(filename/(st+'_1983_2017_dRR_gapless.txt'), 
                         names = ['Year', 'Month', 'Day', 'Precip'], 
                         index_col = 'Year')

    
    Gauge = G_met.groupby('Year').sum('Precip').Precip  
    
    long = dat.iloc[i,1]
    lati = dat.iloc[i,2]
    chirps_gh = gh_data.sel(longitude = long, latitude = lati, method = 'nearest')
    cru_gh = cru_data.sel(lon = long, lat = lati, method = 'nearest')
    
    #convert chirps and cru to dataframe
    conv_chirps_gh = chirps_gh.to_dataframe()
    conv_cru_gh = cru_gh.to_dataframe()

    #select precip from chirps
    precip_chirps = conv_chirps_gh['precip']
    
    precip_gauge = pd.DataFrame(Gauge).set_index(precip_chirps.index)
    
    conv_cru_gh['Stations'] = st
    conv_cru_gh['Chirps_precip'] = precip_chirps
    conv_cru_gh['Gauge_precip'] = precip_gauge
    conv_cru_gh.rename(columns={'pre':'CRU_precip'}, inplace=True)
    conv_cru_gh = pd.DataFrame(conv_cru_gh)

    
    DATA = DATA.append(conv_cru_gh, ignore_index=True)

    
DATA = DATA.set_index('Stations')
DATA = pd.DataFrame(DATA)
DATA

In [ ]:
######## Plot for Rainfall pattern  #######


fig, axes = plt.subplots(nrows = 1, ncols = 3, figsize = (16,20), subplot_kw = {'projection' : ccrs.PlateCarree()})
ax=axes.flatten()
for i in range(3):

    ax[i].add_feature(cf.COASTLINE.with_scale('10m'), linewidth=0.5)
    ax[i].add_feature(cf.BORDERS,linewidth=0.5)
    ax[i].set_extent([-3.4,1.4,11.5,4.5])
    ax[i].set_xticks([-3.2,-2.2,-1.2,0,1.2], crs=ccrs.PlateCarree())
    ax[i].set_yticks([11,10,9,8,7,6,5], crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax[i].xaxis.set_major_formatter(lon_formatter)
    ax[i].yaxis.set_major_formatter(lat_formatter)
    ax[i].set_title(DATA.columns[2+i])

    
    cb = ax[i].scatter(x=DATA['lon'], y=DATA['lat'], c = DATA.iloc[:,2+i], cmap='plasma',linewidths = 5,s=100)

    color_bar = fig.add_axes([0.95, 0.35, 0.025, 0.3])
fig.colorbar(cb, cax=color_bar);


In [ ]:
x = 101
a=range(x)
plt.plot(np.zeros(x),a, c='k')
plt.plot(np.ones(x)*x,a, c='k')
plt.plot(a, np.ones(x)*(x-1), c='k')
plt.plot(a, np.zeros(x)*(x-1), c='k')

plt.plot(np.ones(x)*x/2,a, 'k--')
plt.plot(a,np.ones(x)*x/2, 'k--')


plt.text(40, x+30, 'Gauge events', fontsize=16)
plt.text(20, x+10, 'yes', fontsize=16)
plt.text(80, x+10, 'no', fontsize=16)



plt.text(-50, 40, 'Gridded\nevents', fontsize=16)
plt.text(-30, 20, 'no', fontsize=16)
plt.text(-30, 80, 'yes', fontsize=16)

plt.text(20, x-30, 'hits', fontsize=16)
plt.text(70, x-30, 'false\nalarms', fontsize=16)

plt.text(20, 20,  'misses', fontsize=16)
plt.text(60, 20,  'correct\nnegatives', fontsize=16)



plt.xticks([],[])
plt.yticks([],[])

1. (H/(H+F))*100    #Chances that the event will actually occur?
2. (H/(H+M))*100    # When events occur, how often are the gridded (forecast) correct?

3. Do the forecasts (gridded) predict events too often / not too often enough?
(((H+F)-(H+M))  /(H+M))*100

BIAS
--

In [ ]:
import numpy as np
######################

class stats:
    def standard_error(x):
        ##### standard error function
        try:
            out = np.nanstd(x)/np.sqrt(np.size(x))
        except:
            out = x.nanstd()/np.sqrt(x.size())
        return out
    
    
    def mse(y):
        return stats.standard_error(y)**2
         
    
    #def bias(x,y):
    #    return (y.sum())/(x.sum())
    def bias(x,y):
        return np.nanmean(y) - x
    

    
    ###PoD, FAR, FARate, PC (Percentage of Correctness), NSE (Nash-Sutcliffe Equation)
    ###Hits, Miss, False Alarm, ...
    
    
    def POD(a,b,c):
        H = a.where((a>=c) & b.where(b>=c)).count()
        M = 35 - H
        p = H/(H+M)
        return p
    
    def FAR(s,g,c):
        F = g.where(g>s).count()
        H = s.where((s>=c) & g.where(g>=c)).count()
        FA = F/(H+F)
        return FA
    
    def nse(x,y):
        num = ((y-x)**2).sum()
        den = ((x-np.nanmean(x))**2).sum()
        N = 1 - (num/den)
        return N
    
     #def PC(x,y):
      #  cc = ((x-y)\x)
       # return cc

In [ ]:
all_files = glob.glob('data/*.txt')

fig, axes = plt.subplots(nrows = 11, ncols = 2, figsize = (18,20), sharex=True)
ax = axes.flatten()
for i, file in enumerate(all_files):
    df = pd.read_fwf(file,names = ['Year','Month','Day','Rainfall'],usecols=['Rainfall'])
    df['Date']=pd.date_range(start='1983-01-01', end='2017-12-31',freq='D')
    df=df.set_index('Date').resample('y').sum()
    ax[i].plot(df.index,df)
    ax[i].set_title(file[5:-26]) 
    ax[i].set_ylim(0,2500)
    
    #fig.savefig('Yearly Rainfall patttern updated.pdf',bbox_inches='tight')

In [ ]:
class d:
    def open_file(path="~/Desktop/DATASETS", data=None, fname='/precip.mon.mean.nc'):
        file = xr.open_dataset(path+data.upper()+fname)
        return file
gauge = pd.read_csv('~/Desktop/DATASETS/GAUGE/GMet_location_avgSI.txt', usecols=[0,1,2,4], sep='\s+', header=None)
gauge.columns=['station','lon','lat','zone']
    

In [ ]:
import os
path = os.getcwd()

data_names = ['CRU','GPCC', 'GPCP']#, 'CHIRPS']#,'TAMSAT','ERA5', 'CMORPH', 'PERSIANN', 'IMERG']
filenames = ['cru_ts4.05.1901.2020.pre.dat.nc','GPCC.nc','precip.mon.mean.nc'/\,'chirps_new.nc']

labels = ['Cru','Gpcc','Gpcp']#,'CHIRPS']
fig, axes = plt.subplots(ncols=2, nrows= len(data_names), figsize=(18,10))
plt.subplots_adjust(bottom=0.15, right=0.75, wspace=0.35, hspace=0.5)
data_files = ['data ' +y for y in data_names]
for i, data in enumerate(data_names[:3]): #Datanames Subsetting Here
    gauges = pd.DataFrame(pd.date_range(start='1983-01-01', end='2017-12-31', freq='Y'), columns=['Time'] ).set_index('Time')
    dataset = d.open_file(path='~/Desktop/DATASETS/', data=data, fname='/'+filenames[i])
   
    if data == 'CRU':
        dataset = dataset.pre.sel(lon=slice(-3.5,1.5), lat=slice(4,12), time=slice('1983','2017'))
        
    elif data == 'GPCC':
        dataset  = dataset.precip.sel(lon=slice(-3.5,1.5), lat=slice(12,4), time=slice('1983','2017'))
     
    elif data == 'GPCP':
        dataset  = dataset.precip.sel(lon=slice(-3.5,1.5), lat=slice(12,4), time=slice('1983','2017'))
     

    else:
        dataset = dataset.precip.sel(lon=slice(-3,1.5), lat=slice(4,12), time=slice('1983','2017'))
    dataset = dataset.groupby('time.year').sum('time').rename({'year':'time'})
    #RAI_files[i] = drought_indices.RAI_XD(dataset, 'time')
    
    

In [ ]:
E = xr.open_dataset('~/Desktop/DATASETS/GPCP/precip.mon.mean.nc')
E

In [ ]:
fig, axes = plt.subplots(nrows = 3, ncols = 3, figsize = (16,20), subplot_kw = {'projection' : ccrs.PlateCarree()})
ax=axes.flatten()
for i in range(9):
    ax[i].add_feature(cf.COASTLINE.with_scale('10m'), linewidth=0.5)
    ax[i].add_feature(cf.BORDERS,linewidth=0.5)
    ax[i].set_extent([-3.4,1.4,11.5,4.5])
    ax[i].set_xticks([-3.2,-2.2,-1.2,0,1.2], crs=ccrs.PlateCarree())
    ax[i].set_yticks([11,10,9,8,7,6,5], crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax[i].xaxis.set_major_formatter(lon_formatter)
    ax[i].yaxis.set_major_formatter(lat_formatter)
    ax[i].set_title(file[5:-26])
    dat['Rainfall'] = df.Rainfall

    cb=ax[i].scatter(x=fl['Lon'], y=fl['Lat'], c=fl.iloc[:,2+i], cmap='plasma',linewidths = 5,s=100)
    if i in [0,3,6]:
        fig.colorbar(cb, ax=axes[int(i/3),2])
        plt.scatter

#### import numpy as np
######################

class stats:
    def standard_error(x):
        ##### standard error function
        try:
            out = np.nanstd(x)/np.sqrt(np.size(x))
        except:
            out = x.nanstd()/np.sqrt(x.size())
        return out
    
    
    def mse(y):
        return stats.standard_error(y)**2
         
    
    def bias(x,y):
        return (y.sum())/(x.sum())
    

    
    ###PoD, FAR, FARate, PC (Percentage of Correctness), NSE (Nash-Sutcliffe Equation)
    ###Hits, Miss, False Alarm, ...
    
    
    def pod(a,b,c):
        H = a.where((a>=c) & b.where(b>=c)).count()
        M = 35 - H
        p = H/(H+M)
        return p

In [ ]:
g_data = xr.open_mfdataset('~/Desktop/Datasets/CRU/cru_ts4.05.1901.2020.pre.dat.nc')
g_data